<a href="https://colab.research.google.com/github/zahrababacar6c/Tweeter-Sentiment-Analysis/blob/main/tweeter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import des librairies**

In [ ]:
!pip install transformers

In [ ]:
import os
from transformers import BertForSequenceClassification
import torch
import pandas as pd
from  tqdm.notebook  import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import f1_score
import random

# **Chargemnet & affichage du dataset**

In [ ]:
df  = pd.read_csv("/content/smile-annotations.csv",names=['id','text','category'])
df.set_index('id',inplace=True)

In [ ]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


## **Analyse Exploratoire et Pretraitement des donnees**

In [ ]:
df.text.iloc[3]

'@Sofabsports thank you for following me back. Great to hear from a diverse &amp; interesting panel #DefeatingDepression @RAMMuseum'

In [ ]:
df.category.value_counts()

,count
category,
nocode,1572
happy,1137
not-relevant,214
angry,57
surprise,35
sad,32
happy|surprise,11
happy|sad,9
disgust|angry,7


In [ ]:
#suppression des categories Nocode et les categories multiples
df = df[df.category != 'nocode']
df = df[~df.category.str.contains('\|')]
df.category.value_counts()

,count
category,
happy,1137
not-relevant,214
angry,57
surprise,35
sad,32
disgust,6


In [ ]:
#Labelisataion des differentes Category
labels = df.category.unique()
labels

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [ ]:
label_dict={}
for i ,label in enumerate(labels):
  label_dict[label]=i
  print(i,label)

0 happy
1 not-relevant
2 angry
3 disgust
4 sad
5 surprise


In [ ]:
#creation de la colonne label
df['label'] = df.category.replace(label_dict)
df

<ipython-input-11-e691e63e786d>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df.category.replace(label_dict)


,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0


# **Séparation Entrainement & Validation**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.label.values, test_size=0.15,
                                                  random_state=17, stratify=df.label.values)

In [ ]:
#creation de la colonne data type
df["data_type"]=["not_set"]*df.shape[0]
df.loc[X_train, "data_type"]="train"
df.loc[X_val, "data_type"]="val"
df.groupby(["category","label", "data_type"]).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

# **Tokenization & Encodage des donnees**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
data_train_encoded = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens =True,
    return_attention_mask = True,
    pad_to_max_length=True,
    max_length = 256,
    return_tensors='pt')
input_ids_train =data_train_encoded['input_ids']
attention_masks_train = data_train_encoded["attention_mask"]
labels_train=torch.tensor(df[df.data_type=='train'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
data_val_encoded = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].text.values,
    add_special_tokens =True,
    return_attention_mask = True,
    pad_to_max_length=True,
    max_length = 256,
    return_tensors='pt')
input_ids_val =data_val_encoded['input_ids']
attention_masks_val = data_val_encoded["attention_mask"]
labels_val=torch.tensor(df[df.data_type=='val'].label.values)

In [ ]:
df_train = TensorDataset(input_ids_train,
                              attention_masks_train,labels_train)
df_val = TensorDataset(input_ids_val,
                              attention_masks_val,labels_val)

# **Preparation du Modele BERT**

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states =False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **Creation du DataLoader**

In [ ]:
batch_size  = 32

dataloader_train =DataLoader(
    df_train,
    sampler = RandomSampler(df_train),
    batch_size=batch_size)

In [ ]:
dataloader_val =DataLoader(
    df_val,
    sampler = RandomSampler(df_val),
    batch_size=batch_size)

# **Opimisateur et Scheduler**

In [ ]:
optimizer = AdamW(model.parameters(),lr =2e-5,eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 10
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs)

# **Performance  Metrics**

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat= labels.flatten()
    return f1_score(labels_flat, preds_flat, average = "weighted")

In [ ]:
def accuracy_per_class(preds, labels):
    labeldict_inverse = {v:k for k,v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat= labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class:{labeldict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

# **Training Loop**

In [ ]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch{:1d}'.format(epoch),
                        leave = False,
                        disable=False)
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {
            'input_ids':batch[0],
            'attention_mask':batch[1],
            'labels':batch[2]}

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item() /len(batch))})
        os.makedirs('Models', exist_ok=True)

    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss:{loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1  = f1_score_func(predictions,true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch1:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch {epoch}
Training loss:0.43003322556614876
Validation loss: 0.5196038867746081
F1 Score (weighted): 0.789399036235171


Epoch2:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch {epoch}
Training loss:0.33162373434752224
Validation loss: 0.5608591139316559
F1 Score (weighted): 0.7954544188507924


...


Epoch {epoch}
Training loss:0.07464080911595375
Validation loss: 0.6659901993615287
F1 Score (weighted): 0.8401638343688448


Epoch9:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch {epoch}
Training loss:0.07565354441758246
Validation loss: 0.6667055764368602
F1 Score (weighted): 0.8401638343688448


Epoch10:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch {epoch}
Training loss:0.07701988103799522
Validation loss: 0.6662319472857884
F1 Score (weighted): 0.8401638343688448


# **Chargement et Evaluation du modele**

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.to(device)
pass

In [ ]:
model.load_state_dict(torch.load('Models/BERT_ft_epoch10.model', map_location=torch.device('cpu')))

<ipython-input-34-8403e21c5202>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Models/BERT_ft_epoch10.model', map_location=torch.device('

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_val)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class:happy
Accuracy:164/171

Class:not-relevant
Accuracy:17/32

Class:angry
Accuracy:6/9

Class:disgust
Accuracy:0/1

Class:sad
Accuracy:1/5

Class:surprise
Accuracy:2/5

